In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.6.0'

In [3]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels






Using TensorFlow backend.


In [4]:
ROOT_PATH = '/home/tang/targa/BigSize1/'
train_data_directory = os.path.join(ROOT_PATH, "train")
validation_data_directory = os.path.join(ROOT_PATH, "val")
test_data_directory = os.path.join(ROOT_PATH, "test")
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)
validation_images,validation_labels = load_data(validation_data_directory)

In [5]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL,valL=array(train_labels),array(test_labels),array(validation_labels)
label_encoder = LabelEncoder()

In [6]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)
integer_encoded_valL = label_encoder.fit_transform(valL)

In [7]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)
validationLabels = np_utils.to_categorical(integer_encoded_valL,num_classes = 32)

In [8]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))
valImages = rgb2gray(np.array(validation_images))
train_flat_images = np.reshape(trainImages,(len(trainImages),160*90))
test_flat_images = np.reshape(testImages,(len(testImages),160*90))
val_flat_images = np.reshape(valImages,(len(valImages),160*90))

In [9]:
valImages.shape

(37312, 160, 90)

In [10]:
train_flat_images[0].shape

(14400,)

In [11]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))

print('val images shape:\t\t{}'.format(val_flat_images.shape))

train images shape:		(37312, 14400)
test images shape:		(37312, 14400)
val images shape:		(37312, 14400)


In [12]:
integer_encoded_trL[0]

18

In [13]:
validationLabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
print('Size of:')
print('- Training-set:\t\t{}'.format(len(trainLabels)))
print('- Test-set:\t\t{}'.format(len(testLabels)))
print('- Validation-set:\t{}'.format(len(validation_labels)))

Size of:
- Training-set:		37312
- Test-set:		37312
- Validation-set:	37312


In [15]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.001,dtype = tf.float32)

In [16]:
#命名空间
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,14400],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
    #定义一个dropout的值
    keep_prob = tf.placeholder(tf.float32,name = 'non-dropout-rate')
with tf.name_scope('layer'):
    with tf.name_scope('W1'):
        #第一层
        W1 = tf.Variable(tf.truncated_normal([14400,1000],stddev=0.1),name='W1')#改成截断正太分布
    with tf.name_scope('b1'):
        
        b1 = tf.Variable(tf.zeros([1000])+0.1,name = 'b1')
    with tf.name_scope('out1'):
        
        output1 = tf.matmul(x,W1) + b1
    with tf.name_scope('L1'):
        L1 = tf.nn.tanh(output1)
    with tf.name_scope('L1-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L1_drop = tf.nn.dropout(L1,keep_prob)

    with tf.name_scope('W2'):
        #第二层
        W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1),name='W2')
    with tf.name_scope('b2'):
        b2 = tf.Variable(tf.zeros([2000])+0.2,name='b2')
    with tf.name_scope('output2'):
        output2 = tf.matmul(L1_drop,W2)+b2
    with tf.name_scope('L2'):
        
        L2 = tf.nn.tanh(output2)
    with tf.name_scope('L2_drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L2_drop = tf.nn.dropout(L2,keep_prob)

    with tf.name_scope('W3'):
        #第三层
        W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1),name='W3')
    with tf.name_scope('b3'):
        b3 = tf.Variable(tf.zeros([1000])+0.1,name='b3')
    with tf.name_scope('output3'):   
        output3 = tf.matmul(L2_drop,W3)+b3
    with tf.name_scope('L3'):
        L3 = tf.nn.tanh(output3)
    with tf.name_scope('L3-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L3_drop = tf.nn.dropout(L3,keep_prob)
    with tf.name_scope('W4'):
        # 第四层
        W4 = tf.Variable(tf.truncated_normal([1000,32],stddev=0.1))
    with tf.name_scope('b4'):        
        b4 = tf.Variable(tf.zeros([32])+0.1)
    with tf.name_scope('output4'):
        output4 = tf.matmul(L3_drop,W4)+ b4
    with tf.name_scope('softmax'):
        #预测分类
        prediction = tf.nn.softmax(output4)
with tf.name_scope('loss'):
    #定义一个损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
    #loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('train'):
    #定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    #train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    with tf.name_scope('correct-prediction'):
        #定义一个求准确率的方法
        #如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值
    with tf.name_scope('accuracy'):    
        #求准确率
        #首先把bool值转化成32位的浮点数，然后求平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [17]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [18]:
with tf.Session() as sess:
    sess.run(init)
    #路径和文件名
    write = tf.summary.FileWriter('logs/fourLayerFullyConnectedTargaDropout',sess.graph)
    for epoch in range(1000):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.6})
        
        acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        print('epoch=' + str(epoch) + 'test accuracy=' + str(acc)+'train accuracy='+ str(train_acc))

epoch=0test accuracy=0.21025407train accuracy=0.2108437
epoch=1test accuracy=0.27540737train accuracy=0.27749786
epoch=2test accuracy=0.36848736train accuracy=0.37049744
epoch=3test accuracy=0.4247427train accuracy=0.4275568
epoch=4test accuracy=0.44398585train accuracy=0.44714838
epoch=5test accuracy=0.4610581train accuracy=0.46416703
epoch=6test accuracy=0.474003train accuracy=0.47756752
epoch=7test accuracy=0.47545025train accuracy=0.4798456
epoch=8test accuracy=0.47770154train accuracy=0.48335657
epoch=9test accuracy=0.4820165train accuracy=0.48726952
epoch=10test accuracy=0.48284733train accuracy=0.48810035
epoch=11test accuracy=0.48252574train accuracy=0.48895797
epoch=12test accuracy=0.4870015train accuracy=0.49171847
epoch=13test accuracy=0.4894672train accuracy=0.49410376
epoch=14test accuracy=0.48887756train accuracy=0.49415737
epoch=15test accuracy=0.49169168train accuracy=0.49678388
epoch=16test accuracy=0.49158448train accuracy=0.49656945
epoch=17test accuracy=0.49292454tr

epoch=144test accuracy=0.5280607train accuracy=0.5540845
epoch=145test accuracy=0.53216124train accuracy=0.55834585
epoch=146test accuracy=0.5308212train accuracy=0.5572202
epoch=147test accuracy=0.5307944train accuracy=0.55561215
epoch=148test accuracy=0.52996355train accuracy=0.5564698
epoch=149test accuracy=0.52982956train accuracy=0.5568986
epoch=150test accuracy=0.53025836train accuracy=0.5581582
epoch=151test accuracy=0.530982train accuracy=0.55797064
epoch=152test accuracy=0.530714train accuracy=0.55810463
epoch=153test accuracy=0.5291327train accuracy=0.55633575
epoch=154test accuracy=0.5322148train accuracy=0.55920345
epoch=155test accuracy=0.531518train accuracy=0.55834585
epoch=156test accuracy=0.5311964train accuracy=0.5588015
epoch=157test accuracy=0.5309552train accuracy=0.5601147
epoch=158test accuracy=0.5289451train accuracy=0.5570594
epoch=159test accuracy=0.5311696train accuracy=0.5603827
epoch=160test accuracy=0.5315984train accuracy=0.56062394
epoch=161test accuracy

epoch=287test accuracy=0.5425332train accuracy=0.5806979
epoch=288test accuracy=0.54403406train accuracy=0.5807515
epoch=289test accuracy=0.542292train accuracy=0.5803495
epoch=290test accuracy=0.5426136train accuracy=0.5803227
epoch=291test accuracy=0.54079115train accuracy=0.58021545
epoch=292test accuracy=0.5426136train accuracy=0.5804835
epoch=293test accuracy=0.542292train accuracy=0.5810195
epoch=294test accuracy=0.542024train accuracy=0.5810463
epoch=295test accuracy=0.54272085train accuracy=0.5802423
epoch=296test accuracy=0.542292train accuracy=0.58059067
epoch=297test accuracy=0.5417828train accuracy=0.58013505
epoch=298test accuracy=0.5427476train accuracy=0.5815019
epoch=299test accuracy=0.54298884train accuracy=0.58142155
epoch=300test accuracy=0.5426672train accuracy=0.5815019
epoch=301test accuracy=0.5423724train accuracy=0.5811535
epoch=302test accuracy=0.54079115train accuracy=0.5801887
epoch=303test accuracy=0.5423188train accuracy=0.5812875
epoch=304test accuracy=0.5

epoch=430test accuracy=0.54837584train accuracy=0.59281194
epoch=431test accuracy=0.547679train accuracy=0.5926511
epoch=432test accuracy=0.5493675train accuracy=0.5934284
epoch=433test accuracy=0.5479739train accuracy=0.5935356
epoch=434test accuracy=0.54773265train accuracy=0.59334797
epoch=435test accuracy=0.5465266train accuracy=0.59230274
epoch=436test accuracy=0.5485635train accuracy=0.5929996
epoch=437test accuracy=0.54781306train accuracy=0.5922759
epoch=438test accuracy=0.5486439train accuracy=0.5940716
epoch=439test accuracy=0.546071train accuracy=0.59243673
epoch=440test accuracy=0.54896545train accuracy=0.59415203
epoch=441test accuracy=0.54800063train accuracy=0.5934016
epoch=442test accuracy=0.5481346train accuracy=0.5936696
epoch=443test accuracy=0.5489387train accuracy=0.5938304
epoch=444test accuracy=0.54749143train accuracy=0.593482
epoch=445test accuracy=0.5471698train accuracy=0.59313357
epoch=446test accuracy=0.54751825train accuracy=0.5942592
epoch=447test accurac

epoch=572test accuracy=0.5519136train accuracy=0.6016027
epoch=573test accuracy=0.55124354train accuracy=0.6010935
epoch=574test accuracy=0.5522084train accuracy=0.6007451
epoch=575test accuracy=0.5501179train accuracy=0.60050386
epoch=576test accuracy=0.552396train accuracy=0.6016295
epoch=577test accuracy=0.5496891train accuracy=0.59988743
epoch=578test accuracy=0.55282485train accuracy=0.6015759
epoch=579test accuracy=0.5519672train accuracy=0.6013883
epoch=580test accuracy=0.5504931train accuracy=0.6014687
epoch=581test accuracy=0.55076116train accuracy=0.6015223
epoch=582test accuracy=0.55287844train accuracy=0.6017367
epoch=583test accuracy=0.5524764train accuracy=0.6015223
epoch=584test accuracy=0.55325365train accuracy=0.6020583
epoch=585test accuracy=0.5526908train accuracy=0.6022459
epoch=586test accuracy=0.55287844train accuracy=0.6022191
epoch=587test accuracy=0.55325365train accuracy=0.60243356
epoch=588test accuracy=0.55330724train accuracy=0.6014687
epoch=589test accurac

epoch=714test accuracy=0.55325365train accuracy=0.6075525
epoch=715test accuracy=0.55545133train accuracy=0.60859776
epoch=716test accuracy=0.5566038train accuracy=0.60830295
epoch=717test accuracy=0.556309train accuracy=0.60806173
epoch=718test accuracy=0.5552101train accuracy=0.6081958
epoch=719test accuracy=0.5555853train accuracy=0.6082761
epoch=720test accuracy=0.55617493train accuracy=0.6092142
epoch=721test accuracy=0.55606776train accuracy=0.6084102
epoch=722test accuracy=0.5544061train accuracy=0.60835654
epoch=723test accuracy=0.5548081train accuracy=0.60787416
epoch=724test accuracy=0.5549153train accuracy=0.60859776
epoch=725test accuracy=0.55604094train accuracy=0.6083298
epoch=726test accuracy=0.5549689train accuracy=0.6086246
epoch=727test accuracy=0.5557193train accuracy=0.60835654
epoch=728test accuracy=0.5550493train accuracy=0.6082761
epoch=729test accuracy=0.5545669train accuracy=0.6083298
epoch=730test accuracy=0.55561215train accuracy=0.6084906
epoch=731test accur

epoch=857test accuracy=0.5593375train accuracy=0.61508363
epoch=858test accuracy=0.5582386train accuracy=0.6147084
epoch=859test accuracy=0.5599003train accuracy=0.6154856
epoch=860test accuracy=0.5597395train accuracy=0.61545885
epoch=861test accuracy=0.56083834train accuracy=0.61553925
epoch=862test accuracy=0.5602755train accuracy=0.61524445
epoch=863test accuracy=0.5605971train accuracy=0.6151104
epoch=864test accuracy=0.5593911train accuracy=0.61567324
epoch=865test accuracy=0.5593375train accuracy=0.6151908
epoch=866test accuracy=0.55885506train accuracy=0.6150032
epoch=867test accuracy=0.5593107train accuracy=0.6150568
epoch=868test accuracy=0.56024873train accuracy=0.61596805
epoch=869test accuracy=0.5589623train accuracy=0.61537844
epoch=870test accuracy=0.5590427train accuracy=0.6152712
epoch=871test accuracy=0.5593107train accuracy=0.61537844
epoch=872test accuracy=0.5593375train accuracy=0.615566
epoch=873test accuracy=0.5596323train accuracy=0.61561966
epoch=874test accura

In [19]:
#change learning rate?